In [1]:
from openstack import connection, identity
import copy

In [2]:
def connection_to_cluster(project):
    conn = connection.Connection(auth_url="http://controller:35357/v3",
                                 project_name=project,
                                 project_domain_name="default",
                                 user_domain_name="default",
                                 username="admin",
                                 password="QIP@t4Ay")
    return conn

In [3]:
class OSmap(object):
    
    def __init__ (self, connection='', OS=True, with_flavors=True, with_volumes=True, hypervisor_details=False):
        'Инит функция'
        if OS == True:
            self.__conn = connection
            self.__sl = [i for i in self.__conn.compute.servers(all_tenants=True)]
            self.__hl = [i for i in self.__conn.compute.hypervisors()]
            
            if with_flavors == True:
                self.__fd = self.__get_fl_dict()
                self.dict_flavors = copy.deepcopy(self.__fd)
            
            if with_volumes == True:
                self.__vd = self.__get_vl_dict()
                self.dict_volumes = copy.deepcopy(self.__vd)
                
                
            self.dict_hypervisors = self.__get_hp_dict(details=hypervisor_details)
            self.dict_servers = self.__get_vm_dict(flavors=with_flavors)
            self.list_names  = self.__get_names()
    
    
    def get_vm_by_name(self, vmn=''):
        'Функция возвращает объект ВМ на основании имени'
        for i in self.__sl:
            if i.name == vmn:
                return i 
    
    def get_vm_by_id(self, vmn=''):
        'Функция возвращает объект ВМ на основании ID'
        for i in self.__sl:
            if i.id == vmn:
                return i 
    
        
    
    def __get_hp_dict(self, details=False):
        'Функция возвращает словарь гипервизоров'
        d = {}
        for i in self.__hl:
            if details == True:
                d[i.name]=self.__conn.compute.get_hypervisor(i.id).to_dict()
            else:
                d[i.name]=i.to_dict()
        return d   
           
    def __get_fl_dict(self):
        'Функция возвращает словарь флейворов'
        d = {}
        for i in set([i.flavor['id'] for i in self.__sl]):
            d[i]=self.__conn.compute.get_flavor(flavor=i).to_dict()
        return d
    
    def __get_vl_dict(self):
        'Функция возвращает словарь вольюмов'
        d = {}
        for i in [i for i in self.__conn.block_store.volumes(all_tenants=True)]:
            d[i.id]=i.to_dict()
        return d
    
    
    def __get_names (self):
        'Функция возвращает список имен ВМ'
        a = [i.name for i in self.__sl]
        return a
    
            
    def __get_vm_dict(self,flavors=True):
        'Возвращает словарь с описанием ВМ из OS + cpu & ram'
        d = {}
        for i in self.__sl:
            d[i.name] = self.get_vm_by_name(vmn=i.name).to_dict()
            if flavors == True:
                d[i.name]['cpu'] = self.__fd[i.flavor['id']]['vcpus']
                d[i.name]['ram'] = self.__fd[i.flavor['id']]['ram']
        return d
        

In [3]:
conn = connection_to_cluster('admin')

In [9]:
p = conn.identity.p()
pp = [i for i in p]

In [13]:
pp[0]

openstack.identity.v3.project.Project(is_domain=False, description=Подсистема мониторинга и управления функционированием, enabled=True, id=028441f1f97b4d1a8cdda085be27a44b, parent_id=56d44c3732da4e54b3b3902e4673c9be, domain_id=56d44c3732da4e54b3b3902e4673c9be, name=pmuf)

In [13]:
%%time
x = OSmap(connection_to_cluster('admin'), with_flavors=False, with_volumes=False)

CPU times: user 2.99 s, sys: 92.3 ms, total: 3.09 s
Wall time: 15.8 s


In [14]:
%%time
y = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=False)

CPU times: user 3.48 s, sys: 48.2 ms, total: 3.52 s
Wall time: 17.9 s


In [15]:
%%time
z = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=True)

CPU times: user 5.16 s, sys: 107 ms, total: 5.26 s
Wall time: 34.7 s


In [16]:
%%time
x1 = OSmap(connection_to_cluster('admin'), with_flavors=False, with_volumes=False, hypervisor_details=True)

CPU times: user 4.11 s, sys: 68.5 ms, total: 4.18 s
Wall time: 21.1 s


In [17]:
%%time
y1 = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=False, hypervisor_details=True)

CPU times: user 5.18 s, sys: 31.3 ms, total: 5.21 s
Wall time: 23.8 s


In [4]:
%%time
z1 = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=True, hypervisor_details=True)

CPU times: user 5.03 s, sys: 181 ms, total: 5.21 s
Wall time: 39.1 s


In [9]:
z1.list_names

['ppoz-popd-process-05',
 'ir-app-01-02',
 'ir-app-01-01',
 'ir-srv-01-01',
 'ir-bd-01-01',
 'ir-bd-01-02',
 'ir-app-bal-02',
 'ir-app-bal-01',
 'ps-bd-bal-02',
 'ps-bd-bal-01',
 'make_FreeBSD_great_again',
 'is_efem_vm',
 'is_efem',
 '9.4.gerc_new',
 'egron-elasticsearch-03',
 'migr-node-079',
 'tnx-mon',
 'agent-pvd3',
 'pkurp-worker-022',
 'pkurp-app-039',
 'pkurp-app-038',
 'pkurp-app-037',
 'pkurp-app-036',
 'pkurp-app-035',
 'pkurp-app-034',
 'pkurp-app-033',
 'pkurp-app-032',
 'pkurp-app-031',
 'pkurp-app-030',
 'pkurp-app-029',
 'pkurp-app-028',
 'pkurp-app-027',
 'pkurp-app-026',
 'pkurp-app-025',
 'pkurp-app-024',
 'pkurp-app-023',
 'pkurp-app-022',
 'pkurp-app-021',
 'ppoz-kuvd-migration-service-63',
 'ppoz-kuvd-migration-service-62',
 'ppoz-kuvd-migration-service-61',
 'ppoz-kuvd-migration-service-91',
 'ppoz-kuvd-migration-service-90',
 'ppoz-kuvd-migration-service-89',
 'ppoz-kuvd-migration-service-87',
 'ppoz-kuvd-migration-service-86',
 'ppoz-kuvd-migration-service-83',

In [10]:
vd = z1.dict_volumes
sd = z1.dict_servers

In [15]:
vid = vd['9c5d9579-7a29-4abb-9bd2-05875c84db1d']['attachments'][0]['server_id']

In [17]:
z1.get_vm_by_id(vid).name

'pg-master-standby'

In [48]:
volumes_attached = [i for i in vd.keys() if vd[i]['attachments'] != []]
volumes_notNone_attached = [i for i in vd.keys() if vd[i]['attachments'] != [] \
                        and z1.get_vm_by_id(vd[i]['attachments'][0]['server_id']) is not None]
volumes_dettached = [i for i in vd.keys() if vd[i]['attachments'] == []]
volumes_None_attached = [i for i in vd.keys() if vd[i]['attachments'] != [] \
                        and z1.get_vm_by_id(vd[i]['attachments'][0]['server_id']) is None]

In [56]:
#sd[z1.get_vm_by_id(vid).name]
#vd['2d2e33a4-5c0f-48e4-9b1e-1cb10e264803']

In [54]:
len(vd.keys()), len(volumes_attached), len(volumes_dettached), len(volumes_None_attached), len(volumes_notNone_attached)

(2582, 2487, 95, 26, 2461)

In [43]:
print('Список дисков с кривым аттачем:')
print('')
for v in volumes_None_attached:
    name = vd[v]['name']
    server = z1.get_vm_by_id(vd[v]['attachments'][0]['server_id'])
    size = vd[v]['size']
    vtype = vd[v]['volume_type']
    v_id = vd[v]['id']
    print(server, v_id ,vtype, size, name)
    

Список дисков с кривым аттачем:

None 7ff3d08d-e0f5-4235-a077-373520d4787c 3par 1 test_multipath_4
None 0a429c0b-2a9d-49ca-8040-bf0377df5852 ceph 10 test_27
None b280a70b-3bcc-4128-8714-ae4683ebcbd0 migr-load 150 new_attach_migr-node-126
None 5297ee54-eb43-4475-8f9e-95f4c5d1c648 migr-load 150 new_attach_migr-node-166
None 04a6f5d2-8ac9-47c0-ae97-36992163f704 ceph 5120 pmo-db-fed-03
None ed8e6daf-2690-4bef-8835-04a46820a917 3par 1024 addit-pg-attribute-77
None d5033ea0-03af-4054-baf2-59584aeb9c74 ceph 40 pg-attribute-77
None 83a5ef8b-d934-4ae7-b0c2-14fa39c39610 3par 1024 addit-pg-attribute-75
None d8fd2821-e039-49b3-ad0f-f2cad76f961a ceph 40 pg-attribute-75
None 8d05ec44-5296-4c34-814f-68301511f941 3par 1024 addit-pg-attribute-74
None 4a19ba68-35e1-4e47-8340-012c37fed07f ceph 40 pg-attribute-74
None 627c0378-a9a2-4be2-b371-72e038808c42 ceph 40 pg-attribute-68
None f895f645-6fa3-431e-b8d8-37595db61c1c ceph 40 pg-attribute-66
None 68e3161d-42c9-4440-be83-25d9652d403a ceph 40 pg-attribute-

In [45]:
#vd['68e3161d-42c9-4440-be83-25d9652d403a']

In [49]:
print('Список дисков с аттачем:')
print('')
for v in volumes_notNone_attached:
    name = vd[v]['name']
    server = z1.get_vm_by_id(vd[v]['attachments'][0]['server_id']).name
    size = vd[v]['size']
    vtype = vd[v]['volume_type']
    v_id = vd[v]['id']
    print(server, v_id ,vtype, size, name)

Список дисков с аттачем:

pg-master-standby 9c5d9579-7a29-4abb-9bd2-05875c84db1d 3par 1024 3par_1Tb_pg-master-standby
pg-master 0afa9d9d-39f3-448e-8013-49bd9e183d0d 3par 1000 3par_1Tb_pg-master_03
pg-master ded4ebae-a760-4623-88ae-601789a742a0 3par 1000 3par_1Tb_pg-master_02
pg-master 3e8bf8c6-94f6-448d-8be5-1e6b98f7ce3d 3par 1000 3par_1Tb_pg-master_01
isurz-uploader-020 9ab8fc02-c25e-43a0-aa03-a012148f3079 3par 500 3par_500GB_isurz-uploader-020-01
isurz-uploader-019 2c3b20af-f7d5-423d-a147-c283e045da8c 3par 500 3par_500GB_isurz-uploader-019-01
isurz-uploader-018 469bc149-7356-426f-b99c-8f7495ce7042 3par 500 3par_500GB_isurz-uploader-018-01
isurz-uploader-017 54b1c6c4-c387-49a3-bef4-c5842845ac28 3par 500 3par_500GB_isurz-uploader-017-01
isurz-uploader-016 114ee7d9-3e0b-4a15-a1fa-72bc4e04eaa7 3par 500 3par_500GB_isurz-uploader-016-01
isurz-uploader-015 b9a4f725-4c37-457c-a3b9-de30cb0ef8d7 3par 500 3par_500GB_isurz-uploader-015-01
isurz-uploader-014 0ab417b3-2cad-45bf-9dda-f544c3b38593 3

migr-node-173 9b130c15-506a-43d5-b059-8ac0f53f7450 ceph_alt2_vms 72 migr-node-173
ppoz-db-request-18 2ebfa830-75c8-4227-bf63-20ba033c5e68 3par 1500 ppoz-db-request-18_3par_1.5TB
ppoz-db-request-17 04a947b6-3a6e-46c9-b08f-ee094adf6707 3par 1500 ppoz-db-request-17_3par_1.5TB
ppoz-db-request-16 dab84422-af15-4ebd-b7c4-01d3ac604cb2 3par 1500 ppoz-db-request-16_3par_1.5TB
ppoz-service-validation-06 98b5c418-170c-4542-b7cc-3d2dfc91526c fvolumes 100 fvolumes_100G_sd-1695_ppoz-service-validation-06
ppoz-service-validation-05 355ab0e6-b51e-4bba-a643-1eae7a6e6f26 fvolumes 100 fvolumes_100G_sd-1695_ppoz-service-validation-05
ppoz-service-validation-04 9e0965c1-60ad-4cab-81ad-878005061d59 fvolumes 100 fvolumes_100G_sd-1695_ppoz-service-validation-04
ppoz-service-validation-03 5abca1c2-60cb-4b1f-b70d-beb5f0191600 fvolumes 100 fvolumes_100G_sd-1695_ppoz-service-validation-03
ppoz-service-validation-02 b507391c-629b-49af-8044-d241d576519c fvolumes 100 fvolumes_100G_sd-1695_ppoz-service-validation-02


egron-mgmt-01 feaefc8c-7ee6-4321-a902-1d8d1993f3f3 fvolumes 1024 fvolumes_1T_sd-1540_egron-mgmt-01
migr-grp-oracle-64 7489fc5e-0570-4d1c-8bf8-8401ca3109fe fvolumes 10240 fvolumes_10T_sd-1534_migr-grp-oracle-64
migr-grp-oracle-37 cad38745-1d72-4762-b2ab-3288b36ed03d fvolumes 10240 fvolumes_10T_sd-1534_migr-grp-oracle-37
migr-grp-oracle-40 c4b8c5cd-c9bd-4203-9a8f-f86bdb342dca fvolumes 10240 fvolumes_10T_sd-1526_migr-grp-oracle-40
pkurp-consumer-002 1f9e0d84-426b-4942-83d5-a672c53d609d fvolumes 40 pkurp-consumer-002
pkurp-consumer-001 c95ff6e9-a2cd-4f2d-886f-57a31a4739f0 fvolumes 40 pkurp-consumer-001
migr-gkn-oracle-71 d17a8742-958b-4d6b-8210-fcc05f949201 fvolumes 5120 fvolumes_5T_sd-1503_migr-gkn-oracle-71
migr-init-snap-01 8c88c1b1-99bc-4489-966a-bd494a97ff99 fvolumes 500 migr-init-snap-01
migr-init-04 6342458b-99df-4fcf-933e-9c8b6a36c032 fvolumes 1000 migr-init-04
1007 b4f0e90e-9769-4b59-bf70-b075f6b61223 3par 100 4_100_3par_sd-1493
1007 59479fba-2172-4f55-a7f0-83aeb52af01e 3par 100 3

migr-grp-oracle-91 dc65aa22-809c-4c81-917e-715c32682116 fvolumes 15000 fvol_migr-grp-oracle-91_15000
migr-grp-oracle-90 aa5857b1-5ba6-4d11-ac49-e3dd93644d00 fvolumes 15000 fvol_migr-grp-oracle-90_15000
migr-grp-oracle-89 80534a9e-0c56-49aa-b158-107cba856071 fvolumes 15000 fvol_migr-grp-oracle-89_15000
migr-grp-oracle-87 db1913a7-f2fe-4b4d-b070-f843a921b012 fvolumes 15000 fvol_migr-grp-oracle-87_15000
migr-grp-oracle-86 e53497cf-24d4-408b-8c84-1b1d53b30f7f fvolumes 15000 fvol_migr-grp-oracle-86_15000
migr-grp-oracle-83 1a83229b-3c39-4ed2-a5e3-9fa49b43edea fvolumes 15000 fvol_migr-grp-oracle-83_15000
migr-grp-oracle-79 25d8947d-dc5f-4788-8897-55657e8f1d5e fvolumes 15000 fvol_migr-grp-oracle-79_15000
migr-grp-oracle-75 97bba708-3750-4049-872e-bd1a5cd1772e fvolumes 15000 fvol_migr-grp-oracle-75_15000
migr-grp-oracle-73 df4f8933-6e40-4ec0-a34e-9e82bca68543 fvolumes 15000 fvol_migr-grp-oracle-73_15000
migr-grp-oracle-70 716c845f-859d-4f36-a469-afd3ee2907ae fvolumes 15000 fvol_migr-grp-oracle

isurz-uploader-040 bc830b48-b27f-4dba-8a9c-0b220f04dbaa rvolumes 1024 back_isurz-uploader-040
isurz-uploader-037 659af2e0-b833-4b95-9b63-0db93557f9af rvolumes 1024 back_isurz-uploader-037
isurz-uploader-019 1fb70b39-875f-4a00-b8dd-84a72d43582a rvolumes 1024 back_isurz-uploader-019
isurz-uploader-018 f4b66ef8-1c70-4b1a-9567-20f6dddf95d4 rvolumes 1024 back_isurz-uploader-018
isurz-uploader-014 1c8d3a10-b0b3-4cc2-8abf-3db99eddaa40 rvolumes 1024 back_isurz-uploader-014
isurz-uploader-013 0fd585ee-dc7d-45f3-9af2-08ce0ba2d555 rvolumes 1024 back_isurz-uploader-013
isurz-uploader-006 6d611102-5201-42e9-b601-035853606749 rvolumes 1024 back_isurz-uploader-006
isurz-uploader-002 2f69124a-b8cb-45b0-a944-274f8c4a0a46 rvolumes 1024 back_isurz-uploader-002
isurz-uploader-036 fa0d846a-07c9-4ed6-87ac-ed7358689164 rvolumes 1024 back_isurz-uploader-036
isurz-uploader-034 ce7c62f2-2bdf-496d-bda9-d2cb1257ec7c rvolumes 1024 back_isurz-uploader-034
isurz-uploader-033 35a3033b-da37-4980-a7cc-6ae9f95cf2bb rvol

migr-arc-80 619091ce-58f2-41cd-ab91-600d53628a11 rvolumes 3000 new_attach_migr-arc-80
migr-arc-79 79c411f6-6ce7-4633-b50e-0d1709363578 rvolumes 3000 new_attach_migr-arc-79
migr-arc-78 12ac472f-2834-4882-9f08-6e5a78d8fd9e rvolumes 3000 new_attach_migr-arc-78
migr-arc-77 2c367e7a-3e21-4423-8949-ef319a711b30 rvolumes 3000 new_attach_migr-arc-77
migr-arc-75 cee730cd-560c-4e2f-af47-91399181cc73 rvolumes 3000 new_attach_migr-arc-75
migr-arc-74 19969abc-8703-4724-aa60-58053285b83d rvolumes 3000 new_attach_migr-arc-74
migr-arc-73 7cfdafbc-7506-40a0-b75a-4151e259e3ef rvolumes 3000 new_attach_migr-arc-73
migr-arc-72 327e747f-cb35-4b6d-94b9-757b1b7a0cd7 rvolumes 6000 new_attach_migr-arc-72
migr-arc-70 21927204-f2c6-491b-8187-520ff653d9c1 rvolumes 3000 new_attach_migr-arc-70
migr-arc-68 8920f863-4a66-4639-8981-e425505a8e10 rvolumes 3000 new_attach_migr-arc-68
migr-arc-66 8c89bee5-dd01-473a-b4eb-d615b6790459 rvolumes 3000 new_attach_migr-arc-66
migr-arc-65 4fe4f7b6-7392-46c5-bd48-4f95694c5f51 rvolu

popd-geoapi-balancer-01 2e8f418a-4cd2-4ebf-bd9e-aa01838b07d6 ceph 40 popd-geoapi-balancer-01
migr-gkn-oracle-80 f024a34b-3aac-44c7-a698-1c811310216c ceph 15000 migr-gkn-oracle-80
migr-gkn-oracle-74 9e641ab5-53b1-4ff3-b4d3-d004a8899521 ceph 15000 migr-gkn-oracle-74
migr-gkn-oracle-43 9b34ab2c-3ff7-4f75-8a9f-6a1bf2ce70bc ceph 15000 migr-gkn-oracle-43
migr-grp-oracle-85 2d41db64-81a6-4f35-a605-1a5ceeb05fda ceph 15000 migr-grp-oracle-85
migr-grp-oracle-81 85cf5998-8d02-4395-aeb2-56a63dd0bab3 ceph 15000 migr-grp-oracle-81
migr-grp-oracle-74 dd716084-415d-48c3-914a-22c3754713fb ceph 15000 migr-grp-oracle-74
migr-grp-oracle-43 3c6d2b5a-8e00-40a1-be4c-a44e7218efca ceph 15000 migr-grp-oracle-43
migr-init-01 206d1ff5-fa1c-498e-ae0a-94b67726a120 3par 1024 migr-init-01
isurz-writer-039 3666cffe-993c-4a62-850c-a06b7e7c6b0c ceph 40 re-isurz-writer-039_40
isurz-writer-009 4b312d10-e9e4-411b-a21c-06fdc12d3a6d ceph 40 re-isurz-writer-009_40
popd-bpd-01 cc18923f-aead-47f6-878e-54c0073b94ee 3par 2048 re-

pkurp-worker-008 c4ff8e43-6ad4-4154-90e7-ceb90cf03645 ceph 40 pkurp-worker-008
pkurp-worker-007 ac8766d8-706c-4c08-8d38-57ff4eb090ba ceph 40 pkurp-worker-007
pkurp-worker-006 304c4b40-1a9a-4bc6-ba60-17d3e6c1fb3b ceph 40 pkurp-worker-006
pkurp-worker-005 45df8b18-f385-44bc-b251-384fc3882f6c ceph 40 pkurp-worker-005
pkurp-app-010 043ff03d-1ceb-4f00-b482-1034ba2d4c99 ceph 40 pkurp-app-010
pkurp-app-009 c71b2900-5b8b-4ce4-a25f-e97cead94008 ceph 40 pkurp-app-009
pkurp-app-008 94a03ecc-d215-42b6-8755-d020552853a6 ceph 40 pkurp-app-008
pkurp-app-007 358b9b9c-1ea9-4248-a4e6-93518d40dd39 ceph 40 pkurp-app-007
pkurp-app-006 5c977d7b-23ec-40ab-816c-5a907ecc2ee5 ceph 40 pkurp-app-006
pkurp-app-005 b0befc74-5c46-4c8f-96bc-60513f9ebd78 ceph 40 pkurp-app-005
site-int-04 ed207219-931c-46e6-8553-296d1c871c3e ceph 150 site-int-04
site-int-03 0a4588b6-0b06-4fb9-882c-b7f534f5636d ceph 150 site-int-03
site-solr-04 c0467b3d-5ee6-4745-a87a-1a84e2c8e1c8 ceph 150 site-solr-04
site-solr-03 56a9da0f-0abc-4382-bb

ppoz-pkurp-process-01 5e69bb09-d9db-453b-9d9f-89f1d2338aac ceph 150 ppoz-pkurp-process-01
ppoz-nsi-adapter-02 433663cb-c816-43f9-8340-4b6f35643cc2 ceph 150 ppoz-nsi-adapter-02
ppoz-nsi-adapter-01 b17ea958-e668-444f-b263-492650ee8203 ceph 150 ppoz-nsi-adapter-01
ppoz-kuvd-bal-02 a8a32827-9941-4ac9-b922-412c38a557d2 ceph 150 ppoz-kuvd-bal-02
ppoz-kuvd-bal-01 48a57c81-85f1-4e0d-aee3-992c1d7998c8 ceph 150 ppoz-kuvd-bal-01
ppoz-gmp-services-02 cb70f1ee-daad-4d4b-b481-7e2136af1f33 ceph 150 ppoz-gmp-services-02
ppoz-gmp-services-01 d31b28a8-4b89-4676-baee-2dff62675ba1 ceph 150 ppoz-gmp-services-01
ppoz-gmp-process-02 c91702a4-47d5-4810-b09d-b73458228bb6 ceph 150 ppoz-gmp-process-02
ppoz-gmp-process-01 0c603c7e-9e17-4581-95b7-e8531cf4451b ceph 150 ppoz-gmp-process-01
ppoz-gmp-bal-02 5143d1e8-587e-4e55-902b-262bdb6d938a ceph 150 ppoz-gmp-bal-02
ppoz-gmp-bal-01 f1f9d165-5423-4ed1-9c73-ffd200d37065 ceph 150 ppoz-gmp-bal-01
popd-geoserver-balancer-01 08190a2f-d7bb-47a2-8dc9-e67e36b71b5a ceph 100 p

migr-gkn-oracle-19 6c86a5b3-956c-4b9a-a327-a52ed82ef1e9 ceph 50 migr-gkn-oracle-19
migr-gkn-oracle-18 ce3a8df8-193f-48fe-b04c-d89162af7320 ceph 50 migr-gkn-oracle-18
migr-gkn-oracle-17 422b6806-4e87-45e8-8a54-bf933dc59a61 ceph 50 migr-gkn-oracle-17
migr-gkn-oracle-16 99637720-3688-4356-9841-5179f99b5210 ceph 50 migr-gkn-oracle-16
migr-gkn-oracle-15 6beb95ff-4ccd-421e-a764-ebda96a32900 ceph 50 migr-gkn-oracle-15
migr-gkn-oracle-14 7aa8e6a4-9b58-4d3a-9ccf-b51f918f2fad ceph 50 migr-gkn-oracle-14
migr-gkn-oracle-13 1e725246-dc5e-4c42-bd49-d3c83a872bc8 ceph 50 migr-gkn-oracle-13
migr-gkn-oracle-12 47e2483e-3b56-4b5b-ac12-08d332baef5a ceph 50 migr-gkn-oracle-12
migr-gkn-oracle-11 e3cc0d75-5339-4983-9a79-a5d38ddd962b ceph 50 migr-gkn-oracle-11
migr-gkn-oracle-10 527858cf-2bce-4ef8-946a-dfcc8d9cb005 ceph 50 migr-gkn-oracle-10
migr-jst-oracle-9 3b006b99-7393-48b8-8c4c-729ec421ed8c ceph 50 migr-jst-oracle-9
migr-jst-oracle-7 a23272cc-7c0b-4d9b-a1a5-4cdfeda53731 ceph 50 migr-jst-oracle-7
migr-jst

In [53]:
print('Список дисков без аттача:')
print('')
for v in volumes_dettached:
    name = vd[v]['name']
    #server = z1.get_vm_by_id(vd[v]['attachments'][0]['server_id']).name
    size = vd[v]['size']
    vtype = vd[v]['volume_type']
    v_id = vd[v]['id']
    print(name, v_id ,vtype, size)

Список дисков без аттача:

3par_mult4 dccbe4da-ccde-4360-98da-852e09919e4e 3par 100
3par_mult3 23d3784a-e44b-45a0-9933-e9361fa8209d 3par 100
3par_mult2 4886af1e-c181-487b-ab7b-832391e78c0d 3par 100
3par_mult1 c90d6983-9de0-41c4-a467-157af9b5335e 3par 100
3par_1T_sd-0000_test_migr 8deee8aa-836f-4198-b8dd-60eca9204a7b 3par 1
test_nvme 2616118e-4401-418f-8669-cee3c3d5d80a migr-1 1024
test_fio 38e69928-aeed-410f-9b5e-6e20c8209974 rfiles 50
fvolumes_60GB_sd-2035_isurz-indexer-test-01 f964be76-d0d6-41fb-972c-4b6771ae98ac fvolumes 60
isurz-indexer-test-01_vms2 7cc03ad6-abdb-4c20-bd9e-f3e066b8bca7 ceph_alt2_vms 72
3par_512M_sd-1861_pg-mq-02-standby 43e911b1-9c95-4ca2-b874-4c76fa4e39a4 3par 512
fvol 7a01c6b7-2437-425b-a118-b60dc3149f12 fvolumes 10
3par1 5a795246-f7ee-4d97-b0a1-5d1384ab6a44 3par 10
test_fstring2 27e2a7e3-710f-4e6d-878f-25e144f35011 rvolumes 10
test_fstring a5282932-6366-4ac4-b585-246d1fcc8287 rvolumes 10
test_fstr ce5c2b1f-be23-452a-b714-66d5275c820c ceph_alt2_backup 10
cirros_v

In [ ]:
conn = connection.Connection(user_agent=)